### **1. Gemini 2.5 Pro: 네이티브 오디오 + 이미지 멀티모달 체험 (30분)**

**활동 목표:** 이미지와 음성을 **동시에** 입력받아 상황을 이해하고 답변하는 음성 인식 챗봇을 만듭니다. 이를 통해 Gemini가 단순히 텍스트를 넘어 시청각 정보를 함께 처리하는 '네이티브 멀티모달'을 직접 경험합니다.

**기존 멀티 모달과 네이티브 멀티 모달의 차이**

* 기존 멀티모달: 이미지 인식 모델, 음성 인식 모델, 언어 모델이 각각의 결과를 합치는 방식.

* **네이티브 멀티모달 (Gemini 2.5 Pro):** 하나의 모델이 처음부터 이미지와 오디오 스트림을 함께 받아들여 통합적으로 이해. 마치 우리가 눈으로 보고 귀로 들으며 상황을 파악하는 것과 유사합니다.


# **실습하기**

## **실습 1: 환경 설정**

AI 모델을 우리 코드에서 사용하려면, 먼저 우리 작업 환경(Google Colab)에 몇 가지 준비를 해야 합니다.

### **1. 라이브러리 설치하기**

파이썬 라이브러리는 특정 기능을 쉽게 사용하도록 미리 만들어 놓은 \*\*'도구 상자'\*\*와 같습니다. 우리는 이 도구 상자를 가져와서 사용하기만 하면 됩니다.

  * **왜 필요한가요?**

      * `google-generativeai`: 구글의 Gemini AI와 대화할 수 있게 해주는 **필수 통신 도구**입니다.
      * `Pillow`: AI에게 이미지를 보여주거나, AI가 이미지를 이해할 수 있도록 도와주는 **이미지 처리 도구**입니다.
      * `pyaudio`: 원래는 마이크 같은 오디오 장치를 다룰 때 필요하지만, 우리가 사용할 Google Colab 환경에서는 웹 브라우저의 기본 기능을 사용하므로 **별도로 설치할 필요가 없어 더 편리합니다.**

  * **설치 방법**
    Google Colab의 코드 셀에 아래의 명령어를 그대로 입력하고 실행(▶️ 버튼 클릭 또는 `Shift`+`Enter`)하세요. `!`는 Colab에게 "이건 파이썬 코드가 아니라 시스템 명령어입니다"라고 알려주는 기호입니다. `pip`는 파이썬의 도구 상자(라이브러리) 설치 관리자입니다.

In [ ]:
# 먼저 pip부터 업그레이드해주는 것이 좋습니다.
!pip install --upgrade pip

# 이제 우리가 필요한 라이브러리를 설치합니다. 
# 붉은색 경고 메시지가 나타나도 괜찮습니다!
!pip install -q -U google-generativeai pillow

80607.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


80616.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



    설치가 완료되면 별다른 메시지 없이 조용히 끝나거나, 설치 과정이 잠시 보이다가 완료됩니다. `-q` 옵션은 "quiet"의 약자로 설치 과정을 조용히 진행하라는 의미입니다.

-----

### **2. API 키 인증하기**

API 키 인증은 [**사전 준비사항에서 다루었습니다.**](./PREPARE.md) 혹시 그 단계를 놓쳤거나 확인하고 싶으시다면 아래 가이드를 따라 진행해주세요. API 키는 우리가 Gemini AI를 사용할 자격이 있다는 것을 증명하는 '비밀 열쇠'입니다. 이 열쇠는 다른 사람에게 절대 노출되면 안 됩니다.

Google Colab에는 이런 비밀 키를 안전하게 보관하는 **'보안 비밀(Secrets)'** 기능이 있습니다.

  * **[등록 방법](./PREPARE.md) (사전 준비사항에서 API키를 가져왔다면, 코드에서 키 불러오기 파트를 진행해주세요)**

    1.  Colab 화면 왼쪽 사이드바에서 **열쇠 모양(🔑) 아이콘**을 클릭하세요. '보안 비밀' 탭이 열립니다.

    2.  **`+ 새 보안 비밀 추가`** 버튼을 클릭하세요.

    3.  두 개의 입력 칸이 나타납니다.

          * **이름(Name):** `GOOGLE_API_KEY` 라고 **똑같이** 입력하세요. (대소문자 구분)
          * **값(Value):** 사전 준비 단계에서 Google AI Studio를 통해 발급받아 메모장에 복사해 둔 `AIzaSy...`로 시작하는 긴 문자열을 **붙여넣기** 하세요.

    4.  입력이 끝났으면 오른쪽에 있는 **토글 버튼을 켜서** 이 노트북에서 키를 사용할 수 있도록 활성화해주세요. (버튼이 파란색으로 보이면 활성화된 것입니다.)

  * **코드에서 키 불러오기**
    이제 아래 코드를 실행해서, 방금 등록한 비밀 열쇠를 우리 프로그램으로 가져와 봅시다.

In [ ]:
# 1. 필요한 도구를 불러옵니다.
import google.generativeai as genai
from google.colab import userdata # Colab의 보안 비밀(Secrets) 기능을 사용하기 위한 도구

# 2. 'GOOGLE_API_KEY'라는 이름으로 저장된 비밀 키를 가져옵니다.
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    # 3. 가져온 키를 제미나이 설정에 등록합니다.
    genai.configure(api_key=GOOGLE_API_KEY)
    
    print("✅ API 키가 성공적으로 설정되었습니다! 다음 단계로 진행할 준비가 되었습니다.")

except Exception as e:
    print("🚨 API 키 설정에 실패했습니다. 왼쪽 '🔑' 아이콘에서 API 키의 이름과 값이 정확히 설정되었는지 다시 확인해주세요.")

위 코드 셀을 실행했을 때 "✅ API 키가 성공적으로 설정되었습니다\!"라는 메시지가 나타나면, 모든 환경 설정이 완벽하게 끝난 것입니다.

2. **오디오 및 이미지 입력 함수 제작:**

* 웹캠으로 실시간 이미지를 캡처하는 Python 코드 작성.

* 마이크로 실시간 음성을 녹음하는 Python 코드 작성.


3. **Gemini 2.5 Pro 모델 호출:**

* 캡처된 이미지와 녹음된 음성 데이터를 Gemini 모델에 **동시에** 전달.

* **프롬프트 예시:** "여기 사진 속에 있는 물건 보이지? [이미지 데이터] 이걸로 뭘 할 수 있을지 알려줘. [음성 데이터]"

* 모델이 이미지와 음성 컨텍스트를 종합적으로 이해하고 실시간으로 답변을 생성하는 것을 확인.

4. **결과 확인 및 음성 출력:**

* Gemini가 생성한 텍스트 답변을 화면에 출력.

* (심화) Text-to-Speech(TTS) API를 활용하여 답변을 음성으로 변환해 스피커로 출력.

### **2. Google AI Studio를 활용한 Python API 앱 빌드 (30분)**

**활동 목표:** Google AI Studio의 '코드 가져오기' 기능을 사용하여, 방금 만든 멀티모달 챗봇의 핵심 로직을 재사용 가능한 Python API 코드로 변환하는 방법을 배웁니다.

**Google AI Studio란?**

* Google AI Studio는 단순히 모델을 테스트하는 '놀이터'가 아니라, 아이디어를 실제 코드로 빠르게 전환해주는 '프로토타이핑 툴'입니다. 프롬프트 엔지니어링을 통해 원하는 기능을 구현하고, 즉시 Python, cURL 등 다양한 형태의 코드를 얻을 수 있습니다.

**실습하기**

1. **Google AI Studio 접속:**

* 웹 브라우저에서 [Google AI Studio](https://aistudio.google.com/)에 접속.


2. **프롬프트 작성 및 테스트:**


* '새 프롬프트'를 생성하고, 앞선 실습과 유사한 멀티모달 프롬프트를 작성.

* **시스템 명령어(System Instruction) 설정:** "당신은 사용자가 보여주는 이미지와 음성 질문을 바탕으로 창의적인 아이디어를 제안하는 AI 비서입니다." 와 같이 모델의 역할을 구체적으로 정의.

* 예시 이미지와 텍스트 질문을 입력하며 원하는 결과가 나올 때까지 프롬프트를 수정.


3. **코드 생성 및 추출:**

* 화면 상단의 **`</> 코드 가져오기`** 버튼 클릭.

* Python 탭을 선택하여 생성된 API 호출 코드를 복사.

4. **Python 앱에 통합:**


* Colab 노트북으로 돌아와 복사한 코드를 붙여넣기.

* 이 코드가 어떻게 API 키를 사용하고, 모델을 호출하며, 결과를 파싱하는지 구조를 함께 분석.

* (심화) 이 코드를 기반으로 간단한 Flask 또는 FastAPI 웹 서버의 API 엔드포인트로 만드는 방법을 시연.